In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [3]:
dataset_dir = "H:/Alexnet/Photo(last)"
input_shape = (224, 224, 3)


In [4]:
batch_size = 32
epochs = 10


In [5]:
class_names = sorted(os.listdir(dataset_dir))
num_classes = len(class_names)


In [6]:
df = pd.DataFrame(columns=["filepath", "class"])

for class_index, class_name in enumerate(class_names):
    class_dir = os.path.join(dataset_dir, class_name)
    filenames = os.listdir(class_dir)
    filepaths = [os.path.join(class_dir, filename) for filename in filenames]
    
    df = df.append(pd.DataFrame({"filepath": filepaths, "class": str(class_index)}))

df = df.reset_index(drop=True)


C:\Users\TANJIL\AppData\Local\Temp\ipykernel_15044\3524949522.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({"filepath": filepaths, "class": str(class_index)}))
C:\Users\TANJIL\AppData\Local\Temp\ipykernel_15044\3524949522.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({"filepath": filepaths, "class": str(class_index)}))
C:\Users\TANJIL\AppData\Local\Temp\ipykernel_15044\3524949522.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({"filepath": filepaths, "class": str(class_index)}))
C:\Users\TANJIL\AppData\Local\Temp\ipykernel_15044\3524949522.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pa

C:\Users\TANJIL\AppData\Local\Temp\ipykernel_15044\3524949522.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({"filepath": filepaths, "class": str(class_index)}))
C:\Users\TANJIL\AppData\Local\Temp\ipykernel_15044\3524949522.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({"filepath": filepaths, "class": str(class_index)}))
C:\Users\TANJIL\AppData\Local\Temp\ipykernel_15044\3524949522.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({"filepath": filepaths, "class": str(class_index)}))
C:\Users\TANJIL\AppData\Local\Temp\ipykernel_15044\3524949522.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pa

In [7]:
train_df = df.sample(frac=0.7, random_state=42)
test_df = df.drop(train_df.index)
val_df = test_df.sample(frac=0.5, random_state=42)
test_df = test_df.drop(val_df.index)


In [8]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(dataframe=train_df, x_col="filepath", y_col="class",
                                                    target_size=input_shape[:2], batch_size=batch_size,
                                                    class_mode="categorical", shuffle=True)
test_generator = test_datagen.flow_from_dataframe(dataframe=test_df, x_col="filepath", y_col="class",
                                                  target_size=input_shape[:2], batch_size=batch_size,
                                                  class_mode="categorical", shuffle=False)
val_generator = test_datagen.flow_from_dataframe(dataframe=val_df, x_col="filepath", y_col="class",
                                                 target_size=input_shape[:2], batch_size=batch_size,
                                                 class_mode="categorical", shuffle=False)


Found 1298 validated image filenames belonging to 47 classes.
Found 279 validated image filenames belonging to 47 classes.
Found 278 validated image filenames belonging to 47 classes.


In [9]:
base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=input_shape)
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation="relu"))
model.add(Dense(num_classes, activation="softmax"))


In [10]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(train_generator, steps_per_epoch=train_generator.n // train_generator.batch_size,
          epochs=epochs, validation_data=val_generator, validation_steps=val_generator.n // val_generator.batch_size)


Epoch 1/10
40/40 [==============================] - 379s 9s/step - loss: 2.1237 - accuracy: 0.5608 - val_loss: 0.5596 - val_accuracy: 0.8555
Epoch 2/10
40/40 [==============================] - 336s 8s/step - loss: 0.1855 - accuracy: 0.9494 - val_loss: 0.1620 - val_accuracy: 0.9453
Epoch 3/10
40/40 [==============================] - 316s 8s/step - loss: 0.0945 - accuracy: 0.9660 - val_loss: 0.1059 - val_accuracy: 0.9570
Epoch 4/10
40/40 [==============================] - 326s 8s/step - loss: 0.0836 - accuracy: 0.9652 - val_loss: 0.1650 - val_accuracy: 0.9570
Epoch 5/10
40/40 [==============================] - 312s 8s/step - loss: 0.0730 - accuracy: 0.9684 - val_loss: 0.0799 - val_accuracy: 0.9531
Epoch 6/10
40/40 [==============================] - 310s 8s/step - loss: 0.0734 - accuracy: 0.9708 - val_loss: 0.0956 - val_accuracy: 0.9648
Epoch 7/10
40/40 [==============================] - 308s 8s/step - loss: 0.0557 - accuracy: 0.9731 - val_loss: 0.0917 - val_accuracy: 0.9609
Epoch 8/10
40

In [11]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.n // test_generator.batch_size)
print("Test Accuracy:", test_accuracy)


8/8 [==============================] - 9s 1s/step - loss: 0.1324 - accuracy: 0.9531
Test Accuracy: 0.953125


In [1]:
predictions = model.predict(test_generator, steps=test_generator.n // test_generator.batch_size)


NameError: name 'model' is not defined

In [ ]:
import numpy as np
from PIL import Image

# Load the image
image_path ="H:/1_0_3127.png"
img = Image.open(image_path)
img = img.resize((224, 224))  # Resize the image to match the input size of VGG16

# Convert the image to RGB mode
img = img.convert("RGB")

# Preprocess the image
img_array = np.array(img)
img_array = img_array / 255.0  # Normalize the pixel values
img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension

# Make the prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)

print("Predicted class:", class_names[predicted_class])
